In [1]:
# Dependencies

import gmaps
import numpy as np
import pandas as pd
import requests
import time
from datetime import date
from scipy.stats import linregress
from matplotlib import pyplot as plt
from config import (gkey)

In [2]:
#import dataset
property_value = pd.read_csv('Draft.csv')
len(property_value)


18525

In [3]:
property_value = property_value.dropna()
property_value = property_value.loc[property_value['State']=="CA"]
len(property_value)

1072

In [4]:
property_value.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1/31/14,...,6/30/20,7/31/20,8/31/20,9/30/20,10/31/20,11/30/20,12/31/20,1/31/21,2/28/21,3/31/21
12,97564,13,94109,Zip,CA,CA,San Francisco,San Francisco-Oakland-Hayward,San Francisco County,716780.0,...,870882,867306,865503,864815,857989,852618,841552,833637,824861,820867
19,96107,22,90250,Zip,CA,CA,Hawthorne,Los Angeles-Long Beach-Anaheim,Los Angeles County,382463.0,...,564738,569115,577003,583048,586983,591031,595064,601593,608374,614481
41,96027,44,90046,Zip,CA,CA,Los Angeles,Los Angeles-Long Beach-Anaheim,Los Angeles County,452491.0,...,619591,624558,629664,634896,636372,638530,640997,644307,647755,650469
83,97711,88,94501,Zip,CA,CA,Alameda,San Francisco-Oakland-Hayward,Alameda County,348254.0,...,533582,533451,536829,542350,547908,553665,558060,559572,562250,566397
89,97565,94,94110,Zip,CA,CA,San Francisco,San Francisco-Oakland-Hayward,San Francisco County,769165.0,...,1057838,1049245,1044429,1039913,1031005,1021449,1010388,1002934,995229,993192


In [5]:
cagr = []

In [6]:
property_value['CAGR'] = (property_value['3/31/21'] - property_value['1/31/14'])/property_value['1/31/14']*100
property_value_df = pd.DataFrame(property_value)
property_value_df = property_value_df.sort_values(by='CAGR', ascending=False)


In [7]:
property_value_df_sample = property_value.sample(50)
property_value_df_sample.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1/31/14,...,7/31/20,8/31/20,9/30/20,10/31/20,11/30/20,12/31/20,1/31/21,2/28/21,3/31/21,CAGR
1319,97558,1401,94103,Zip,CA,CA,San Francisco,San Francisco-Oakland-Hayward,San Francisco County,698251.0,...,887295,882991,878069,869104,859316,845783,834482,821962,813054,16.441509
14255,97406,18169,93667,Zip,CA,CA,Tollhouse,Fresno,Fresno County,167954.0,...,210069,209288,208722,208467,205941,203207,200029,196781,194732,15.943651
11983,98584,14332,96055,Zip,CA,CA,Los Molinos,Red Bluff,Tehama County,86837.0,...,100451,103039,106467,109828,111963,112637,113197,113629,114679,32.062370
2096,97488,2250,93940,Zip,CA,CA,Monterey,Salinas,Monterey County,378049.0,...,500598,503574,506133,510346,514165,517051,519213,522485,529366,40.025764
4473,96926,4856,92583,Zip,CA,CA,San Jacinto,Riverside-San Bernardino-Ontario,Riverside County,99749.0,...,178908,180782,180998,182725,184414,187112,189228,191554,194886,95.376395


In [8]:
# # import api_key to match latitudes and longitudes
# from config import api_key

# # creating url for latitude and longitude search
# # Save config information
# url = "http://api.openweathermap.org/data/2.5/weather?"

# # Build query URL
# query_url = url + "appid=" + api_key + "&q="


In [9]:
# # convert city to list
# city_list = property_value_df_selec['City'].to_list()

In [10]:
property_value_df_sample['Lat'] = ""
property_value_df_sample['Lng'] = ""

In [11]:
# create a params dict that will be updated with new city each iteration
params = {"key": gkey}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in property_value_df_sample.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    zipcode = row['RegionName']

    # update address key value
    params['address'] = (zipcode)

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
    # convert to json
    cities_lat_lng = cities_lat_lng.json()

    property_value_df_sample.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
    property_value_df_sample.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]

In [12]:
    property_value_df_sample

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1/31/14,...,9/30/20,10/31/20,11/30/20,12/31/20,1/31/21,2/28/21,3/31/21,CAGR,Lat,Lng
1319,97558,1401,94103,Zip,CA,CA,San Francisco,San Francisco-Oakland-Hayward,San Francisco County,698251.0,...,878069,869104,859316,845783,834482,821962,813054,16.441509,37.7726,-122.41
14255,97406,18169,93667,Zip,CA,CA,Tollhouse,Fresno,Fresno County,167954.0,...,208722,208467,205941,203207,200029,196781,194732,15.943651,36.9349,-119.348
11983,98584,14332,96055,Zip,CA,CA,Los Molinos,Red Bluff,Tehama County,86837.0,...,106467,109828,111963,112637,113197,113629,114679,32.062370,40.078,-122.061
2096,97488,2250,93940,Zip,CA,CA,Monterey,Salinas,Monterey County,378049.0,...,506133,510346,514165,517051,519213,522485,529366,40.025764,36.576,-121.822
4473,96926,4856,92583,Zip,CA,CA,San Jacinto,Riverside-San Bernardino-Ontario,Riverside County,99749.0,...,180998,182725,184414,187112,189228,191554,194886,95.376395,33.8148,-116.95
10755,98082,12568,95258,Zip,CA,CA,Woodbridge,Stockton-Lodi,San Joaquin County,223683.0,...,317701,313202,308724,304950,300801,295655,288307,28.890886,38.1674,-121.314
1070,97016,1129,92780,Zip,CA,CA,Tustin,Los Angeles-Long Beach-Anaheim,Orange County,247275.0,...,347390,349798,352943,356681,359326,361615,362368,46.544535,33.7372,-117.814
109,96025,115,90044,Zip,CA,CA,Los Angeles,Los Angeles-Long Beach-Anaheim,Los Angeles County,216754.0,...,447795,449544,451849,455552,459914,462370,463315,113.751534,33.9479,-118.294
5488,96943,5995,92606,Zip,CA,CA,Irvine,Los Angeles-Long Beach-Anaheim,Orange County,371491.0,...,457884,461335,466277,472038,477061,482240,487833,31.317582,33.6913,-117.822
1538,97135,1642,93117,Zip,CA,CA,Goleta,Santa Maria-Santa Barbara,Santa Barbara County,371804.0,...,418473,420513,422786,428009,433599,442415,453141,21.876311,34.4302,-119.87


In [13]:
#CONFIGURE GMAPS
gmaps.configure(api_key=gkey)

# Store 'Lat' and 'Lng' into  locations 
locations = property_value_df_sample[["Lat", "Lng"]].astype(float)

# humity to plot heat map
CAGR_percent = property_value_df_sample["CAGR"].astype(float)

In [14]:
# humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=CAGR_percent, 
                                 dissipating=False, max_intensity=300,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))